# 智谱视频生成 API 测试 (CogVideoX-Flash - 非对话模型，不支持思考模式)

本 Notebook 用于测试智谱 AI 的视频生成功能，包括文生视频和图生视频。

**模型信息：**
- 模型: CogVideoX-Flash
- 规格: 4K/10秒/60fps
- 特点: AI音效、高清画质、免费

## 1. 环境配置

In [ ]:
# 安装必要的依赖
!pip install requests -q

In [ ]:
import os
import time
import json
import requests
from typing import Optional, Dict, Any, List
from datetime import datetime
from IPython.display import Video, Image, display, HTML

print("✅ 环境配置完成")

In [ ]:
# API 配置
API_KEY = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4"

# 请求头
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

print("✅ API 配置完成")

## 2. 视频生成类封装

In [ ]:
class ZhipuVideoGenerator:
    """智谱视频生成器"""
    
    def __init__(self, api_key: str = API_KEY, base_url: str = BASE_URL):
        self.api_key = api_key
        self.base_url = base_url
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
    
    def submit_task(
        self, 
        prompt: str, 
        image_url: Optional[str] = None,
        size: str = "3840x2160"
    ) -> Dict[str, Any]:
        """
        提交视频生成任务
        
        Args:
            prompt: 视频描述提示词
            image_url: 首帧图片 URL（图生视频时使用）
            size: 分辨率，默认 3840x2160
        
        Returns:
            包含 task_id 的响应数据
        """
        url = f"{self.base_url}/videos/generations"
        
        payload = {
            "model": "cogvideox-flash",
            "prompt": prompt,
            "size": size
        }
        
        if image_url:
            payload["image_url"] = image_url
        
        response = requests.post(url, headers=self.headers, json=payload)
        response.raise_for_status()
        return response.json()
    
    def query_result(self, task_id: str) -> Dict[str, Any]:
        """
        查询任务结果
        
        Args:
            task_id: 任务 ID
        
        Returns:
            任务状态和结果
        """
        url = f"{self.base_url}/async-result/{task_id}"
        
        response = requests.get(url, headers={"Authorization": f"Bearer {self.api_key}"})
        response.raise_for_status()
        return response.json()
    
    def generate_video(
        self,
        prompt: str,
        image_url: Optional[str] = None,
        size: str = "3840x2160",
        max_retries: int = 60,
        poll_interval: int = 5,
        verbose: bool = True
    ) -> Dict[str, Any]:
        """
        生成视频（包含轮询等待）
        
        Args:
            prompt: 视频描述提示词
            image_url: 首帧图片 URL（图生视频时使用）
            size: 分辨率
            max_retries: 最大轮询次数
            poll_interval: 轮询间隔（秒）
            verbose: 是否打印详细信息
        
        Returns:
            包含视频 URL 的结果
        """
        # 提交任务
        if verbose:
            print("📝 提交视频生成任务...")
            
        submit_result = self.submit_task(prompt, image_url, size)
        task_id = submit_result.get("id")
        
        if not task_id:
            raise ValueError(f"提交任务失败: {submit_result}")
        
        if verbose:
            print(f"✅ 任务已提交，ID: {task_id}")
            print(f"⏳ 等待视频生成中...（预计1-3分钟）")
        
        # 轮询查询结果
        start_time = time.time()
        for attempt in range(max_retries):
            time.sleep(poll_interval)
            
            result = self.query_result(task_id)
            status = result.get("task_status")
            
            if status == "SUCCESS":
                elapsed_time = time.time() - start_time
                if verbose:
                    print(f"✅ 视频生成完成！耗时: {elapsed_time:.1f}秒")
                result["elapsed_time"] = elapsed_time
                result["task_id"] = task_id
                return result
            elif status == "FAIL":
                raise RuntimeError(f"视频生成失败: {result}")
            else:
                if verbose:
                    elapsed = time.time() - start_time
                    print(f"   [{elapsed:.0f}s] 轮询 {attempt + 1}/{max_retries}: 状态 {status}")
        
        raise TimeoutError(f"视频生成超时，任务ID: {task_id}")
    
    def display_video(self, result: Dict[str, Any], width: int = 640):
        """
        在 Notebook 中显示视频
        
        Args:
            result: 视频生成结果
            width: 显示宽度
        """
        if result.get("task_status") != "SUCCESS":
            print("❌ 视频生成未成功")
            return
        
        video_result = result.get("video_result", [])
        if not video_result:
            print("❌ 未找到视频结果")
            return
        
        video_url = video_result[0].get("url")
        cover_url = video_result[0].get("cover_image_url")
        
        print(f"🎬 视频 URL: {video_url}")
        print(f"🖼️ 封面 URL: {cover_url}")
        
        # 显示视频播放器
        display(Video(url=video_url, width=width, html_attributes="controls"))

# 初始化生成器
generator = ZhipuVideoGenerator()
print("✅ 视频生成器初始化完成")

## 3. 文生视频基础测试

In [ ]:
# 基础文生视频测试
basic_prompt = """
一只可爱的柴犬在樱花树下追逐蝴蝶，阳光透过花瓣洒落，
形成梦幻的光影效果，微风轻拂，樱花瓣缓缓飘落。
画面采用温暖的色调，充满春日的生机与活力。
"""

print("📝 提示词:")
print(basic_prompt)
print("\n" + "="*50 + "\n")

try:
    result_basic = generator.generate_video(basic_prompt)
    generator.display_video(result_basic)
except Exception as e:
    print(f"❌ 错误: {e}")

## 4. 不同场景测试

### 4.1 动物场景

In [ ]:
# 动物场景 - 海洋生物
animal_prompt = """
深海中，一只巨大的蓝鲸缓缓游过，阳光透过海面形成光束照射下来，
照亮了周围成群的热带鱼和飘动的海草。
水泡缓缓上升，形成梦幻般的海底世界。
"""

print("🐋 动物场景测试")
print(f"提示词: {animal_prompt[:50]}...")

try:
    result_animal = generator.generate_video(animal_prompt)
    generator.display_video(result_animal)
except Exception as e:
    print(f"❌ 错误: {e}")

### 4.2 自然风景

In [ ]:
# 自然风景 - 山水
nature_prompt = """
清晨的黄山云海，金色的阳光从云层中穿透而出，
照在奇松怪石上，云雾在山间流动，如同仙境一般。
远处山峰若隐若现，营造出宁静而壮丽的氛围。
"""

print("🏔️ 自然风景测试")
print(f"提示词: {nature_prompt[:50]}...")

try:
    result_nature = generator.generate_video(nature_prompt)
    generator.display_video(result_nature)
except Exception as e:
    print(f"❌ 错误: {e}")

### 4.3 城市景观

In [ ]:
# 城市景观 - 未来都市
city_prompt = """
未来主义城市夜景，摩天大楼之间穿梭着流线型的飞行汽车，
建筑表面覆盖着动态全息广告和霓虹灯带，
雨滴落在透明玻璃幕墙上，折射出五光十色的光芒，
营造出赛博朋克的氛围。
"""

print("🏙️ 城市景观测试")
print(f"提示词: {city_prompt[:50]}...")

try:
    result_city = generator.generate_video(city_prompt)
    generator.display_video(result_city)
except Exception as e:
    print(f"❌ 错误: {e}")

### 4.4 艺术创作

In [ ]:
# 艺术创作 - 水墨风格
art_prompt = """
水墨画风格的江南烟雨，小桥流水人家，
淡淡的墨色晕染开来，柳枝随风轻摆，
一叶扁舟缓缓划过平静的水面，
远处的山峦在薄雾中若隐若现，
整个画面充满诗意和东方美学意境。
"""

print("🎨 艺术创作测试")
print(f"提示词: {art_prompt[:50]}...")

try:
    result_art = generator.generate_video(art_prompt)
    generator.display_video(result_art)
except Exception as e:
    print(f"❌ 错误: {e}")

## 5. 图生视频测试

In [ ]:
# 图生视频测试
# 使用一张示例图片作为首帧

# 示例图片 URL（这里使用一张风景图片）
# 注意：请使用可访问的图片 URL
image_url = "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=1920&q=80"

# 图生视频的提示词 - 描述画面的动态效果
image_to_video_prompt = """
画面中的山峰在晨光中逐渐明亮，
云层在山间缓缓流动，
阳光透过云隙洒在山脊上，形成金色的光芒，
整个场景呈现出时间的流逝感。
"""

print("🖼️ 图生视频测试")
print(f"首帧图片: {image_url}")
print(f"提示词: {image_to_video_prompt[:50]}...")

# 显示首帧图片
print("\n首帧预览:")
display(Image(url=image_url, width=640))

print("\n开始生成视频...")
try:
    result_image = generator.generate_video(
        prompt=image_to_video_prompt,
        image_url=image_url
    )
    generator.display_video(result_image)
except Exception as e:
    print(f"❌ 错误: {e}")

## 6. 批量生成测试

In [ ]:
# 批量生成测试
# 定义多个提示词进行批量测试

batch_prompts = [
    {
        "name": "科幻场景",
        "prompt": "太空站内部，宇航员漂浮在零重力环境中，透过巨大的观景窗可以看到地球缓缓旋转，星辰闪烁，充满未来科技感。"
    },
    {
        "name": "美食特写",
        "prompt": "精美的日式料理摆盘，寿司上的鱼片新鲜透亮，蒸汽缓缓升起，酱油滴落的瞬间，画面精致而富有食欲。"
    },
    {
        "name": "运动场景",
        "prompt": "冲浪者在巨大的海浪上滑行，水花四溅，阳光照射下水珠晶莹剔透，展现出极限运动的力量与美感。"
    }
]

print("🔄 批量生成测试")
print(f"共 {len(batch_prompts)} 个任务\n")

batch_results = []

for idx, item in enumerate(batch_prompts, 1):
    print(f"\n{'='*50}")
    print(f"任务 {idx}/{len(batch_prompts)}: {item['name']}")
    print(f"{'='*50}")
    
    try:
        result = generator.generate_video(item["prompt"])
        batch_results.append({
            "name": item["name"],
            "result": result,
            "status": "success"
        })
        print(f"✅ {item['name']} 完成")
    except Exception as e:
        batch_results.append({
            "name": item["name"],
            "error": str(e),
            "status": "failed"
        })
        print(f"❌ {item['name']} 失败: {e}")
    
    # 批次间延迟，避免触发限流
    if idx < len(batch_prompts):
        print("\n等待 5 秒后继续...")
        time.sleep(5)

# 汇总结果
print("\n" + "="*50)
print("批量生成结果汇总")
print("="*50)

success_count = sum(1 for r in batch_results if r["status"] == "success")
failed_count = len(batch_results) - success_count

print(f"✅ 成功: {success_count}/{len(batch_results)}")
print(f"❌ 失败: {failed_count}/{len(batch_results)}")

for r in batch_results:
    status_icon = "✅" if r["status"] == "success" else "❌"
    print(f"  {status_icon} {r['name']}")

## 7. 结果展示

In [ ]:
# 展示所有成功的视频结果
print("🎬 视频结果展示\n")

all_results = [
    ("基础测试", locals().get("result_basic")),
    ("动物场景", locals().get("result_animal")),
    ("自然风景", locals().get("result_nature")),
    ("城市景观", locals().get("result_city")),
    ("艺术创作", locals().get("result_art")),
    ("图生视频", locals().get("result_image")),
]

for name, result in all_results:
    if result and result.get("task_status") == "SUCCESS":
        print(f"\n{'='*60}")
        print(f"📹 {name}")
        print(f"{'='*60}")
        
        video_result = result.get("video_result", [])[0]
        video_url = video_result.get("url")
        cover_url = video_result.get("cover_image_url")
        elapsed_time = result.get("elapsed_time", 0)
        
        print(f"⏱️ 生成耗时: {elapsed_time:.1f}秒")
        print(f"🔗 视频链接: {video_url[:60]}...")
        
        # 显示封面
        if cover_url:
            display(Image(url=cover_url, width=320))
        
        # 显示视频播放器
        display(Video(url=video_url, width=480, html_attributes="controls"))
    else:
        print(f"⏭️ {name}: 未执行或未成功")

## 8. 错误处理测试

In [ ]:
# 错误处理测试
print("🧪 错误处理测试\n")

# 测试 1: 空提示词
print("测试 1: 空提示词")
try:
    result = generator.generate_video("")
except Exception as e:
    print(f"✅ 正确捕获错误: {type(e).__name__}")
    print(f"   错误信息: {e}\n")

# 测试 2: 无效的图片 URL
print("测试 2: 无效的图片 URL")
try:
    result = generator.generate_video(
        prompt="测试提示词",
        image_url="https://invalid-url.com/image.jpg"
    )
except Exception as e:
    print(f"✅ 正确捕获错误: {type(e).__name__}")
    print(f"   错误信息: {e}\n")

# 测试 3: 超长提示词
print("测试 3: 超长提示词（>1000字）")
long_prompt = "这是一段测试文字。" * 200
try:
    result = generator.generate_video(long_prompt)
    print(f"✅ 请求已提交（可能会因内容限制而失败）\n")
except Exception as e:
    print(f"✅ 正确捕获错误: {type(e).__name__}")
    print(f"   错误信息: {e}\n")

print("✅ 错误处理测试完成")

## 9. 超时处理测试

In [ ]:
# 超时处理测试
print("⏱️ 超时处理测试\n")

# 测试：设置较短的超时时间
print("测试: 设置较短的超时时间 (max_retries=2)")
print("预期结果: 应该触发 TimeoutError\n")

try:
    # 提交一个正常任务，但设置很短的轮询次数
    result = generator.generate_video(
        prompt="一只小猫在阳光下玩耍",
        max_retries=2,      # 最多轮询 2 次
        poll_interval=3     # 每 3 秒轮询一次
    )
except TimeoutError as e:
    print(f"✅ 正确触发超时: {e}")
except Exception as e:
    print(f"ℹ️ 其他结果: {type(e).__name__}: {e}")

print("\n" + "="*50)
print("超时处理测试说明")
print("="*50)
print("""
在实际使用中，建议：
- max_retries: 60-120 次（5-10 分钟超时）
- poll_interval: 5-10 秒
- 视频生成通常需要 1-3 分钟
- 根据网络状况调整轮询间隔
""")

## 10. 总结

In [ ]:
# 生成测试总结
print("📊 测试总结\n")
print("="*60)

test_items = [
    "✅ 环境配置",
    "✅ 视频生成类封装",
    "✅ 文生视频基础测试",
    "✅ 不同场景测试（动物、自然、城市、艺术）",
    "✅ 图生视频测试",
    "✅ 批量生成测试",
    "✅ 结果展示",
    "✅ 错误处理测试",
    "✅ 超时处理测试",
]

for item in test_items:
    print(item)

print("\n" + "="*60)
print("关键要点")
print("="*60)

summary = """
1. 异步调用流程
   - 提交任务 → 获取 task_id → 轮询查询 → 获取结果

2. 关键参数
   - model: cogvideox-flash
   - prompt: 详细描述场景、主体、动作、氛围
   - image_url: 可选，用于图生视频
   - size: 3840x2160 (4K)

3. 轮询策略
   - poll_interval: 建议 5-10 秒
   - max_retries: 建议 60-120 次
   - 实际生成时间: 1-3 分钟

4. 注意事项
   - 视频 URL 有时效性，请及时下载
   - 提示词越详细，生成效果越好
   - 虽然是免费模型，但注意调用频率限制
"""

print(summary)

print("="*60)
print("测试完成！")
print("="*60)